In [5]:
import xml.etree.ElementTree as ET
import csv

def kml_to_csv(kml_file, csv_file):
    # Parse the KML file
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Correct namespace
    ns = {'': 'http://earth.google.com/kml/2.1'}

    # Open a CSV file for writing
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write CSV headers
        csvwriter.writerow(['Coordinates'])

        # Find the "Track Points" folder
        for folder in root.findall('.//{http://earth.google.com/kml/2.1}Folder', ns):
            name = folder.find('{http://earth.google.com/kml/2.1}name', ns)
            if name is not None and name.text == 'Track Points':
                # Iterate through each Placemark within the "Track Points" folder
                for placemark in folder.findall('{http://earth.google.com/kml/2.1}Placemark', ns):
                    coordinates = placemark.find('.//{http://earth.google.com/kml/2.1}coordinates', ns).text

                    # Write data to CSV
                    csvwriter.writerow([coordinates])
                    
# Convert KML to CSV
kml_to_csv('20240124Caminar al aire libre.kml', 'walk1.csv')


In [8]:
import xml.etree.ElementTree as ET
import csv

def kml_to_csv(kml_file, csv_file):
    # Parse the KML file
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Correct namespace
    ns = {'': 'http://earth.google.com/kml/2.1'}

    # Open a CSV file for writing
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write CSV header
        csvwriter.writerow(['Longitude', 'Latitude'])

        # Find the "Track Points" folder
        for folder in root.findall('.//{http://earth.google.com/kml/2.1}Folder', ns):
            name = folder.find('{http://earth.google.com/kml/2.1}name', ns)
            if name is not None and name.text == 'Track Points':
                # Iterate through each Placemark within the "Track Points" folder
                for placemark in folder.findall('{http://earth.google.com/kml/2.1}Placemark', ns):
                    coordinates = placemark.find('.//{http://earth.google.com/kml/2.1}coordinates', ns).text

                    # Split the coordinates into longitude, latitude, and altitude (optional)
                    lon, lat, *_ = map(float, coordinates.split(','))

                    # Write data to CSV
                    csvwriter.writerow([lon, lat])

# Convert KML to CSV
kml_to_csv('20240124Caminar al aire libre.kml', 'output.csv')


In [9]:
!pip install folium 

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/18/09/8569904c8ce5679cc02826d98de633c07abcd2443a23181e5f71ff9dacbc/folium-0.15.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/2f/e7/603b136221de923055716d23e3047da71f92e0d8ba2c4517ce49a54fe768/branca-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 915.7 kB/s eta 0:00:001m534.6 kB/s eta 0:00:01


In [1]:
import folium
import pandas as pd

# Load your CSV file
data = pd.read_csv('output.csv')

In [2]:
# Create a map centered around an average location
map_center = [data['Latitude'].mean(), data['Longitude'].mean()]
map_bordeaux = folium.Map(location=map_center, zoom_start=13)

# Add points to the map
for idx, row in data.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=str(idx)).add_to(map_bordeaux)

# Display the map
map_bordeaux


In [4]:
import folium
import pandas as pd

# Load your CSV file
data = pd.read_csv('output.csv')
S
# Create a map centered around an average location
map_center = [data['Latitude'].mean(), data['Longitude'].mean()]
map_bordeaux = folium.Map(location=map_center, zoom_start=13)

# Create a FeatureGroup for the PolyLine
line_layer = folium.FeatureGroup(name='Line Layer').add_to(map_bordeaux)

# Add a PolyLine to the line layer
folium.PolyLine(locations=data[['Latitude', 'Longitude']].values, color='blue').add_to(line_layer)

# Create a FeatureGroup for the Markers
marker_layer = folium.FeatureGroup(name='Marker Layer').add_to(map_bordeaux)

# Add points to the marker layer with Popup showing coordinates
for idx, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Coordinates: {row['Latitude']}, {row['Longitude']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_layer)

# Add LayerControl to toggle visibility
folium.LayerControl(collapsed=False).add_to(map_bordeaux)

# Display the map
map_bordeaux

In [ ]:
# Load your CSV files
data1 = pd.read_csv('output1.csv')  # Replace 'output1.csv' with the actual file path
data2 = pd.read_csv('output2.csv')  # Replace 'output2.csv' with the actual file path
data3 = pd.read_csv('output3.csv')  # Replace 'output3.csv' with the actual file path

# Create a map centered around an average location
map_center = [data1['Latitude'].mean(), data1['Longitude'].mean()]
map_comparison = folium.Map(location=map_center, zoom_start=13)

# Create FeatureGroups for each dataset
line_layer1 = folium.FeatureGroup(name='GT - Line').add_to(map_comparison)
marker_layer1 = folium.FeatureGroup(name='GT - Markers').add_to(map_comparison)

line_layer2 = folium.FeatureGroup(name='GPS - Line').add_to(map_comparison)
marker_layer2 = folium.FeatureGroup(name='GPS - Markers').add_to(map_comparison)

line_layer3 = folium.FeatureGroup(name='Teria GNSS- Line').add_to(map_comparison)
marker_layer3 = folium.FeatureGroup(name='Teria GNSS - Markers').add_to(map_comparison)

# Add PolyLines for each dataset
folium.PolyLine(locations=data1[['Latitude', 'Longitude']].values, color='blue').add_to(line_layer1)
folium.PolyLine(locations=data2[['Latitude', 'Longitude']].values, color='green').add_to(line_layer2)
folium.PolyLine(locations=data3[['Latitude', 'Longitude']].values, color='red').add_to(line_layer3)

# Add Markers for each dataset with Popup showing coordinates
for idx, row in data1.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"GT - Coordinates: {row['Latitude']}, {row['Longitude']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_layer1)

for idx, row in data2.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"GPS - Coordinates: {row['Latitude']}, {row['Longitude']}",
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(marker_layer2)

for idx, row in data3.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Teria GNSS - Coordinates: {row['Latitude']}, {row['Longitude']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_layer3)

# Add LayerControl to toggle visibility
folium.LayerControl(collapsed=False).add_to(map_comparison)

# Display the map
map_comparison